# **Physics Informed Neural Networks**

Physics-Informed Neural Networks (PINNs) are a class of machine learning models that integrate the physical laws governing a system, typically in the form of partial differential equations (PDEs), into the neural network's training process. This approach allows the network to learn solutions to PDEs without needing a large dataset of solution points. Instead, the PINN enforces the PDEs as constraints in the loss function, guiding the network to discover solutions that satisfy the underlying physics. This makes PINNs particularly useful for solving complex problems in fluid dynamics, heat transfer, and other fields where data may be sparse or hard to obtain. 


## Model

This notebook defines a simple PINN model using TensorFlow. The class `PINN` constructs a fully connected neural network with layers specified by `layers_dims`. Each layer uses the `tanh` activation function, except the output layer, which produces a single continuous value. The model is designed to take inputs (such as spatial and time coordinates) and return a prediction, which can be further trained to satisfy the given physical laws expressed as PDEs.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import h5py


class PINN(Model):
    def __init__(self, layers_dims):
        super(PINN, self).__init__()
        self.network = tf.keras.Sequential()
        for dim in layers_dims:
            self.network.add(layers.Dense(dim, activation="tanh"))
        self.network.add(layers.Dense(1))  

    def call(self, inputs):
        return self.network(inputs)

## Physics informed loss

A function to compute the loss for training a Physics-Informed Neural Network (PINN) to solve a partial differential equation (PDE), while enforcing initial conditions, boundary conditions, and the PDE itself.
- **Operations** : 
  1. **Gradient Calculation**: 
     - Computes the first and second derivatives of the model's output (`u`) with respect to spatial (`x`) and time (`t`) using `tf.GradientTape`.
  2. **PDE Residual**: 
     - Calculates the residual of the PDE, involving terms like `u_t`, `u_x`, and `u_xx`, which should ideally be zero for a valid solution.

- **Loss Components**:
  - **Initial Condition Loss**: 
    - Measures the difference between the model's predicted solution at `t=0` and the given `initial_condition`.
  - **Periodic Boundary Condition Loss**: 
    - Enforces periodic boundary conditions by ensuring the solution and its spatial derivative are the same at the boundary points (`x_min` and `x_max`).
  - **Residual Loss**: 
    - Penalizes deviations from the PDE's residual to ensure the model satisfies the physical equation.

- **Output**:
  - Returns the total loss, which is the sum of the initial condition loss, periodic boundary loss, and residual loss.

In [2]:
def physics_informed_loss(
    model, x, t, initial_condition, boundary_condition, du, epsilon
):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        epsilon_tensor = tf.fill(x.shape,  tf.constant(epsilon, dtype=tf.float32))
        u = model(tf.concat([x, t, epsilon_tensor], axis=1))

        u_t = tape.gradient(u, t)
        u_x = tape.gradient(u, x)
        u_xx = tape.gradient(u_x, x)

    residual = u_t + u * u_x - epsilon * u_xx

    # Initial condition loss
    initial_loss = tf.reduce_mean(
        tf.square(u[tf.equal(t, tf.reduce_min(t))] - initial_condition)
    )

    # Periodic boundary condition loss
    periodic_loss = tf.reduce_mean(
        tf.square(u[tf.equal(x, tf.reduce_min(x))] - u[tf.equal(x, tf.reduce_max(x))])
    ) + tf.reduce_mean(
        tf.square(
            u_x[tf.equal(x, tf.reduce_min(x))] - u_x[tf.equal(x, tf.reduce_max(x))]
        )
    )

    # Residual loss
    residual_loss = tf.reduce_mean(tf.square(residual))

    return initial_loss + periodic_loss + residual_loss

# Training Loop


- **Operations**:
  1. **Grid Creation**: 
     - Creates a mesh grid of spatial (`x_grid`) and time (`t_grid`) values, then flattens them into 1D arrays (`x_flat` and `t_flat`).
  2. **Optimizer Setup**: 
     - Uses the Adam optimizer with the specified `learning_rate` to minimize the loss.
  3. **Training Loop**: 
     - For each epoch, computes the loss using `physics_informed_loss` and updates the model parameters by computing gradients and applying them through the optimizer.

In [3]:
def train_model(
    model,
    x,
    t,
    initial_condition,
    boundary_condition,
    du,
    epsilon,
    epochs,
    learning_rate,
):
    x_grid, t_grid = tf.meshgrid(x[:, 0], t[:, 0])  
    x_flat = tf.reshape(x_grid, [-1, 1])  
    t_flat = tf.reshape(t_grid, [-1, 1]) 

    optimizer = tf.keras.optimizers.Adam(learning_rate)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = physics_informed_loss(
                model,
                x_flat,
                t_flat,
                initial_condition,
                boundary_condition,
                du,
                epsilon,
            )
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy():.6f}")

## Data Loading


Opens the HDF5 file and retrieves:
- `initial_condition` and `boundary_condition`: Initial and boundary values for the solution, clean or noisy depending on input.
- `clean_data`: The ground truth solution data.
- `du`: Derivative of the solution.
- `epsilon`: A parameter (e.g., diffusion coefficient).
- `u0`: Initial value of the solution.
- `x` and `t`: Spatial and time coordinates.
  

In [4]:
def load_data(file_name, type):
    with h5py.File(file_name, "r") as f:
        initial_condition = f[f"0/initial_condition_{type}"][:]
        boundary_condition = f[f"0/boundary_condition_{type}"][:]
        clean_data = f["0/clean"][:]
        du = f["0/du"][()]
        epsilon = f["0/epsilon"][()]
        u0 = f["0/u0"][()]
        x = f["coords/x-coordinates"][:]
        t = f["coords/t-coordinates"][:-1]
    return (
        initial_condition,
        boundary_condition,
        clean_data,
        du,
        epsilon,
        u0,
        x,
        t,
    )

## Model Setup

This code loads the simulation data from an HDF5 file and prepares it for training a Physics-Informed Neural Network (PINN) to solve the noisy version of the Burgers' equation.

  1. Load data from the file `simulation_data.h5` using the `load_data` function (loading initial conditions, boundary conditions, solution data, and other parameters).
  2. Convert the `x` and `t` coordinate arrays to TensorFlow tensors and reshapes them as column vectors.
  3. Convert `initial_condition` and `boundary_condition` to TensorFlow tensors.
  
- **Model Setup**: 
  - The PINN architecture is initialized with 3 hidden layers, each with 50 neurons (`layers_dims = [50, 50, 50]`).

In [5]:
# Load the data
file_name = (
    "/kaggle/input/burgers-noisy/simulation_data.h5"  # Replace with your data file
)
initial_condition, boundary_condition, clean_data, du, epsilon, u0, x, t = load_data(
    file_name, "noisy"
)

x = tf.convert_to_tensor(x, dtype=tf.float32)[:, None]  # x-coordinates as column vector
t = tf.convert_to_tensor(t, dtype=tf.float32)[:, None]  # t-coordinates as column vector
initial_condition = tf.convert_to_tensor(initial_condition, dtype=tf.float32)
boundary_condition = tf.convert_to_tensor(boundary_condition, dtype=tf.float32)

layers_dims = [50, 50, 50]
epochs = 10000
learning_rate = 1e-3

model = PINN(layers_dims)

## Training Noisy

In [6]:
train_model(
    model,
    x,
    t,
    initial_condition,
    boundary_condition,
    du,
    epsilon,
    epochs,
    learning_rate,
)

Epoch 0, Loss: 1.494818
Epoch 100, Loss: 0.554695
Epoch 200, Loss: 0.329819
Epoch 300, Loss: 0.261835
Epoch 400, Loss: 0.251033
Epoch 500, Loss: 0.243114
Epoch 600, Loss: 0.235674
Epoch 700, Loss: 0.225637
Epoch 800, Loss: 0.210556
Epoch 900, Loss: 0.192006
Epoch 1000, Loss: 0.179609
Epoch 1100, Loss: 0.172866
Epoch 1200, Loss: 0.168310
Epoch 1300, Loss: 0.166577
Epoch 1400, Loss: 0.165576
Epoch 1500, Loss: 0.164757
Epoch 1600, Loss: 0.165785
Epoch 1700, Loss: 0.163406
Epoch 1800, Loss: 0.162896
Epoch 1900, Loss: 0.162403
Epoch 2000, Loss: 0.161876
Epoch 2100, Loss: 0.161491
Epoch 2200, Loss: 0.161090
Epoch 2300, Loss: 0.160681
Epoch 2400, Loss: 0.160311
Epoch 2500, Loss: 0.159940
Epoch 2600, Loss: 0.159551
Epoch 2700, Loss: 0.159180
Epoch 2800, Loss: 0.158714
Epoch 2900, Loss: 0.158261
Epoch 3000, Loss: 0.157784
Epoch 3100, Loss: 0.157315
Epoch 3200, Loss: 0.156725
Epoch 3300, Loss: 0.156198
Epoch 3400, Loss: 0.155630
Epoch 3500, Loss: 0.155062
Epoch 3600, Loss: 0.154460
Epoch 3700, L

## Prediction

#### Boundary Condition enforcement

This function enforces periodic boundary conditions on the solution array `u`.

  - Uses `tf.tensor_scatter_nd_update` to swap the values at the first and last indices of the solution array `u`, ensuring periodicity.

- Returns the modified solution array `u` with periodic boundary conditions applied, preserving the original shape.

In [7]:
def apply_periodic_bc(u, boundary_condition):
    """
    Enforce periodic boundary conditions on the solution.
    Args:
        u: Solution array at a given time step (tensor).
        boundary_condition: Precomputed boundary values (not directly used here).
    Returns:
        Solution array with periodic boundary applied (same shape as input).
    """
    u = tf.tensor_scatter_nd_update(
        u,
        indices=[[0], [-1]], 
        updates=[u[-1], u[0]],  
    )
    return u

#### Iterative time step prediction

  1. Initialize the solution with the `initial_condition` and store the first prediction.
  2. For each time step, concatenates the current spatial coordinates, solution, and `epsilon` to form the input for the model.
  3. The model predicts the solution at the next time step (`u_next`), and periodic boundary conditions are applied using the `apply_periodic_bc` function.
  4. Appends each new prediction to the list `all_predictions`.


In [8]:
def predict_solution_iterative(
    model, x, initial_condition, boundary_condition, epsilon, n_steps
):
    """
    Iteratively predict the solution over time.
    Args:
        model: Trained PINN model.
        x: Spatial coordinates as a tensor.
        initial_condition: Initial condition at t=0.
        boundary_condition: Boundary conditions (periodic).
        epsilon: Diffusion parameter.
        n_steps: Number of time steps to predict.
    Returns:
        Array of predictions at each time step.
    """
    u = tf.convert_to_tensor(initial_condition, dtype=tf.float32)[:, None] 
    all_predictions = [u.numpy().reshape(-1)]
    print(u.shape)
    for _ in range(n_steps):
        epsilon_tensor = tf.fill(
            x.shape, tf.constant(epsilon, dtype=tf.float32)
        )  
        u_input = tf.concat([x, u], axis=1) 
        u_input = tf.concat(
            [u_input, epsilon_tensor], axis=1
        )  
        u_next = model(u_input)[:, 0]  
        u_next = apply_periodic_bc(u_next, boundary_condition)

        all_predictions.append(u_next.numpy())
        u = u_next[:, None]
    return np.array(all_predictions)

#### Predictions 

Prepare the data and make predictions over loaded clean data

In [9]:
initial_condition, boundary_condition, clean_data, du, epsilon, u0, x, t = load_data(
    file_name, "clean"
)
x = tf.convert_to_tensor(x, dtype=tf.float32)[:, None]  
t = tf.convert_to_tensor(t, dtype=tf.float32)[:, None]  
initial_condition = tf.convert_to_tensor(initial_condition, dtype=tf.float32)
boundary_condition = tf.convert_to_tensor(boundary_condition, dtype=tf.float32)
n_steps = 200
predicted_solution = predict_solution_iterative(
    model,
    x=x,
    initial_condition=initial_condition,
    boundary_condition=boundary_condition,
    epsilon=epsilon,
    n_steps=n_steps,
)

print("Predicted solution shape:", predicted_solution.shape)

(1024, 1)
Predicted solution shape: (201, 1024)


## Visualisation of Solution

  1. For each time step, it plots the spatial coordinates (`xcrd`) versus the solution at that step and stores the plot in `ims` for animation.
  2. Uses `ArtistAnimation` to create the animation from the collected frames.
  
  3. Saves the animation as a GIF using `PillowWriter` with a frame rate of 15 fps, and stores it at the specified `path`. The function also closes the figure after saving.

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm


def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(
                xcrd, data[i].squeeze(), animated=True, color="blue"
            ) 
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)
    
visualize_burgers(x, predicted_solution, "noisy_trained_predicted_solution.gif")

100%|██████████| 201/201 [00:00<00:00, 353.87it/s]


## Training Clean

Training over clean data for comparison

In [11]:
file_name = (
    "/kaggle/input/burgers-noisy/simulation_data.h5"  
)
initial_condition, boundary_condition, clean_data, du, epsilon, u0, x, t = load_data(
    file_name, "clean"
)

x = tf.convert_to_tensor(x, dtype=tf.float32)[:, None] 
t = tf.convert_to_tensor(t, dtype=tf.float32)[:, None]  
initial_condition = tf.convert_to_tensor(initial_condition, dtype=tf.float32)
boundary_condition = tf.convert_to_tensor(boundary_condition, dtype=tf.float32)

layers_dims = [50, 50, 50]
epochs = 10000
learning_rate = 1e-3

model = PINN(layers_dims)

In [12]:
train_model(
    model,
    x,
    t,
    initial_condition,
    boundary_condition,
    du,
    epsilon,
    epochs,
    learning_rate,
)

Epoch 0, Loss: 0.800827
Epoch 100, Loss: 0.454979
Epoch 200, Loss: 0.179035
Epoch 300, Loss: 0.167613
Epoch 400, Loss: 0.161234
Epoch 500, Loss: 0.153584
Epoch 600, Loss: 0.138297
Epoch 700, Loss: 0.115942
Epoch 800, Loss: 0.096874
Epoch 900, Loss: 0.090514
Epoch 1000, Loss: 0.083550
Epoch 1100, Loss: 0.080961
Epoch 1200, Loss: 0.077984
Epoch 1300, Loss: 0.075869
Epoch 1400, Loss: 0.073421
Epoch 1500, Loss: 0.071164
Epoch 1600, Loss: 0.069516
Epoch 1700, Loss: 0.069805
Epoch 1800, Loss: 0.068203
Epoch 1900, Loss: 0.066629
Epoch 2000, Loss: 0.065586
Epoch 2100, Loss: 0.064821
Epoch 2200, Loss: 0.064292
Epoch 2300, Loss: 0.063304
Epoch 2400, Loss: 0.062630
Epoch 2500, Loss: 0.064358
Epoch 2600, Loss: 0.061491
Epoch 2700, Loss: 0.060974
Epoch 2800, Loss: 0.060545
Epoch 2900, Loss: 0.060758
Epoch 3000, Loss: 0.063131
Epoch 3100, Loss: 0.059417
Epoch 3200, Loss: 0.059131
Epoch 3300, Loss: 0.058837
Epoch 3400, Loss: 0.058589
Epoch 3500, Loss: 0.058327
Epoch 3600, Loss: 0.058077
Epoch 3700, L

#### Prediction over clean data

In [13]:
initial_condition, boundary_condition, clean_data, du, epsilon, u0, x, t = load_data(
    file_name, "clean"
)
x = tf.convert_to_tensor(x, dtype=tf.float32)[:, None]  
t = tf.convert_to_tensor(t, dtype=tf.float32)[:, None]  
initial_condition = tf.convert_to_tensor(initial_condition, dtype=tf.float32)
boundary_condition = tf.convert_to_tensor(boundary_condition, dtype=tf.float32)
n_steps = 200
predicted_solution = predict_solution_iterative(
    model,
    x=x,
    initial_condition=initial_condition,
    boundary_condition=boundary_condition,
    epsilon=epsilon,
    n_steps=n_steps,
)

print("Predicted solution shape:", predicted_solution.shape)

(1024, 1)
Predicted solution shape: (201, 1024)


#### Visualization

In [14]:
visualize_burgers(x, predicted_solution, "clean_trained_predicted_solution.gif")

100%|██████████| 201/201 [00:00<00:00, 256.87it/s]
